# 章節 15：維度處理

## 15.9 肚臍物種資料

關於獅子、老虎跟熊的問題已經談夠了。我們回到肚臍物種的問題。我們先來看看資料的長相，這是對主體 B1242 做 400 次取樣觀察到 61 種種類的結果：
<pre>
92, 53, 47, 38, 15, 14, 12, 10, 8, 7, 7, 5, 5,
4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
<pre>

有少數的優勢物種佔了很大的比例，大部分物種只有出現一次。這些單獨的物種表明了有些看不見的物種。在獅子老虎的範例中，我們假設每種動物被觀察到的機率是相同的。同樣在肚臍物種資料上，我們假設每種細菌被發現的機率也是相同的。

在實際情況下，收集資料的每個步驟都有可能產生偏差(bias)。

有些種類更可能被棉花棒收集到，或產生可識別的擴增子（amplicons，分子生物學專有名詞）。所以當我們要說每個細菌的流行程度，我們要了解這種來源造成的偏差。

作者承認不是嚴謹的用「物種」(species)這個詞。首先，細菌物種沒有很好的定義。第二，有些序列（read）識別特定物種，其他只識別屬（genus）。為了更精確，這邊應該用「可操作性分類單元」（Operational Taxonomic Unit），簡稱 OTU。

現在我們來處理一些細菌資料吧。先定義一個 Subject 類別來代表每個被研究的主體：

<pre>
class Subject(object):
    def __init__(self, code):
        self.code = code
        self.species = []
</pre>

每個主體有一個字串代碼，例如「B1242」，還有一序列的 (數量，物種名稱) 的 tuple, 依據數量由小排到大。Subject 類別提供數個方法存取數量和物種名稱。詳細請參考 http://thinkbayes.com/species.py

Subject 提供一個方法叫 Process 用來產生跟更新 Species5 物件，此物件包含 Dirichlet 的參數跟物種流行程度。Suite2 提供 DistOfN 方法來計算 n 的後驗分佈。

<pre>
# class Suite2
    def DistN(self):
        items = zip(self.ns, self.probs)
        pmf = thinkbayes.MakePmfFromItems(items)
        return pmf
</pre>

In [2]:
#TODO

上圖顯示對於 B1242 主體物種數量 n 的分佈。剛好 61 個物種的機率趨近於零，並且沒有 0 種物種。最有可能是 72 種種類，90% 的可信區間在 66 到 79種。不太可能有超過 87 種物種。

接著我們計算每個物種的後驗分佈。Species2 提供 DistOfPrevalence 方法：

<pre>
# class Species2
    def DistOfPrevalence(self, index):
        metapmf = thinkbayes.Pmf()

        for n, prob in zip(self.ns, self.probs):
            beta = self.MarginalBeta(n, index)
            pmf = beta.MakePmf()
            metapmf.Set(pmf, prob)
        mix = thinkbayes.MakeMixture(metapmf)
        return metapmf, mix
</pre>

index 為我們想知道的物種索引。對每個 n 我們有不同物種流行的後驗分佈。所以迭代所有的 n 跟機率。對每個 n 計算其邊際分佈用 Beta 物件表示。Beta 物件只有包含 alpha 和 beta 參數；他們不是一個 Pmf，但有提供 MakePmf 方法產生一個離散趨近連續 beta 分佈的 Pmf 物件。

metapmf 包含對不同 n 的流行程度的分佈。MakeMixture 將所有的不同 n 的流行程度的分佈結合產生一個單一個流行程度的分佈。

In [1]:
#TODO

上圖顯示最多出現的五個物種。這些最流行的物種們佔了 400 次定序的 23%，但因為幾乎確定有沒捕捉到的物種，因此其最有可能的流行程度是 20%，90% 的可信區間在 17% 到 23%。